In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict



from langchain.embeddings import HuggingFaceBgeEmbeddings
model_name = "bge-large-zh-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    cache_folder="./bge-large-zh-v1.5",
    encode_kwargs=encode_kwargs,
    query_instruction="为这个句子生成表示以用于检索相关文章："
)
model.query_instruction = "为这个句子生成表示以用于检索相关文章："


#embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")
#vector_store = InMemoryVectorStore(embeddings)
print("ok")

ok


In [ ]:
# 测试 embedding 模型

# 从本地缓存加载模型
from langchain.embeddings import HuggingFaceBgeEmbeddings
model_name = "bge-large-zh-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    cache_folder="./bge-large-zh-v1.5",
    encode_kwargs=encode_kwargs,
    query_instruction="为这个句子生成表示以用于检索相关文章："
)
model.query_instruction = "为这个句子生成表示以用于检索相关文章："
print("ok")

In [12]:
# https://python.langchain.com/docs/tutorials/rag/ 中的例子
# 创建一个 app
from dotenv import load_dotenv
load_dotenv(override=True)

import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.vectorstores import InMemoryVectorStore

from langchain.embeddings import HuggingFaceBgeEmbeddings
import os
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent

# 从博客中加载文章
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    # 使用 BeautifulSoup 筛选出文章内容
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()
# 文章有42K
print(f"Total characters: {len(docs[0].page_content)}")

DeepSeek_API_KEY = os.getenv("DEEPSEEK_API_KEY")
llm = init_chat_model(model="deepseek-chat", model_provider="deepseek")

embeddings = HuggingFaceBgeEmbeddings(
    model_name="bge-large-zh-v1.5",
    cache_folder="./bge-large-zh-v1.5",
)

vector_store = InMemoryVectorStore(embeddings)

# 文章有42K，对大多数模型来说，会超过context window限制
# 将文章分块，用于 embeding 和 vector store
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)
# 被分成了大概63个sub-documents
print(f"Split blog post into {len(all_splits)} sub-documents.")

# 对 sub-documents 进行vector store，用于在运行时检索
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
# N.B. for non-US LangSmith endpoints, you may need to specify
# api_url="https://api.smith.langchain.com" in hub.pull.
# langchain预置的prompt模板
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

response = graph.invoke({"question": "What is Task Decomposition?"})
print(response["answer"])

Total characters: 43047
Split blog post into 63 sub-documents.


/Users/xiangfei.cui/workspace/codes/github/study/langchain/venv/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Task decomposition is the process of breaking down a complex task into smaller, manageable steps or subgoals. It can be achieved through methods like simple prompting with LLMs, task-specific instructions, or human input. Additionally, approaches like LLM+P use external planners for long-horizon planning, while techniques like Chain of Thought and Tree of Thoughts enhance decomposition by guiding step-by-step reasoning.


In [ ]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

load_dotenv(override=True)
model = init_chat_model(model="deepseek-chat", model_provider="deepseek")
question = "你好，请介绍一下你自己"
result = model.invoke(question)
print(result.content)

<class 'langchain_deepseek.chat_models.ChatDeepSeek'>
你好！我是 **DeepSeek Chat**，由深度求索公司（DeepSeek）研发的智能 AI 助手。我可以帮助你解答各种问题，包括学习、工作、编程、生活百科、创意写作等。以下是我的一些特点：  

✨ **免费使用**：目前无需付费，你可以随时向我提问！  
📚 **知识丰富**：我的知识截止到 **2024 年 7 月**，可以为你提供最新的科技、新闻、学术等资讯。  
📝 **多格式支持**：可以阅读并分析 **PDF、Word、Excel、PPT、TXT** 等文件，帮助你提取关键信息。  
💡 **逻辑与创意兼具**：无论是数学计算、代码编写，还是写诗、写故事，我都能胜任！  
📏 **超长上下文**：支持 **128K** 上下文，能记住更长的对话内容，适合处理复杂问题。  

你可以问我任何问题，比如：  
- **学习**：如何高效复习？这道数学题怎么做？  
- **工作**：如何写一份优秀的简历？帮我优化 PPT。  
- **编程**：Python 代码调试、算法讲解、项目建议。  
- **生活**：推荐旅游景点、健身计划、美食食谱。  

试试向我提问吧，我会尽力帮助你！😊


In [ ]:
# 测试langchain hub
from langchain import hub
from dotenv import load_dotenv
load_dotenv(override=True)
prompt = hub.pull("rlm/rag-prompt")
print(prompt)

/Users/xiangfei.cui/workspace/codes/github/study/langchain/venv/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]
